# Demo Transmitter Program

To be used with transducers in a small container of water. Doesn't use a full modulated signal, instead communicates with clicks.

### Imports and Microblaze Setup

In [5]:
from pynq.overlays.base import BaseOverlay
base = BaseOverlay("base.bit")

In [6]:
from pynq.lib import Pmod_ADC, Pmod_DAC
import time
import ctypes
import numpy as np
import array
from pynq import allocate
import numpy as np 
import matplotlib.pyplot as plt 


In [7]:
dac = Pmod_DAC(base.PMODB)

### Transmitter

Encodes ones and zeros in the length of gaps between clicks. A longer gap means one, while a shorter means zero. Each message will start and end with a click to bookend the bits.

In [9]:
def send_bits(bit_list):
    fs = 168000
    freq = 42000
    buffer_len = 40000
    samples_per_cycle = fs/freq
    samples_before_switch = int(samples_per_cycle/2)
    
    cycle_samples = [800]*samples_before_switch + [0]*samples_before_switch
    print(cycle_samples)
    iterations = int(buffer_len / len(cycle_samples)) + 1
    cycle_samples = cycle_samples*iterations
    
    py_buffer = allocate(shape=(buffer_len,), dtype=np.int32)
    
    # now the one that inverts it
    py_buffer_inverted = allocate(shape=(buffer_len,), dtype=np.int32)
    
    # And one that is zero for good measure
    py_buffer_zero = allocate(shape=(buffer_len,), dtype=np.int32)

    for i in range(buffer_len):
        py_buffer[i] = cycle_samples[i]
        py_buffer_inverted[i] = 1600 - cycle_samples[i]
        py_buffer_zero[i] = 0
            
    
    def send_zero_bit():
        dac.write(py_buffer.physical_address|0x00000003)
        dac.write(py_buffer_inverted.physical_address|0x00000003)
        dac.write(py_buffer_zero.physical_address|0x00000003)
        dac.write(py_buffer_zero.physical_address|0x00000003)
        dac.write(py_buffer_zero.physical_address|0x00000003)
    
    def send_one_bit():
        dac.write(py_buffer.physical_address|0x00000003)
        dac.write(py_buffer_inverted.physical_address|0x00000003)
        dac.write(py_buffer_zero.physical_address|0x00000003)
        dac.write(py_buffer_zero.physical_address|0x00000003)
        dac.write(py_buffer_zero.physical_address|0x00000003)
        dac.write(py_buffer_zero.physical_address|0x00000003)
        dac.write(py_buffer_zero.physical_address|0x00000003)
        dac.write(py_buffer_zero.physical_address|0x00000003)
    
    for bit in bit_list:
        if bit:
            send_one_bit()
        else:
            send_zero_bit()
    dac.write(py_buffer.physical_address|0x00000003)
    dac.write(py_buffer_inverted.physical_address|0x00000003)

In [12]:
send_bits([0,1,1,0,0,1,1,0,0,1,1,0,1,1,1,1,0,1,1,0,1,1,1,1,0,1,1,0,0,1,0,0,0,0,1,1,1,1,1,1])

[800, 800, 0, 0]


In [18]:
send_bits([0,1,1,0,1,0,0,0,0,1,1,0,0,1,0,1,0,1,1,0,1,1,0,0,0,1,1,0,1,1,0,0,0,1,1,0,1,1,1,1,0,0,1,0,0,0,0,0,0,1,1,1,0,1,1,1,0,1,1,0,1,1,1,1,0,1,1,1,0,0,1,0,0,1,1,0,1,1,0,0,0,1,1,0,0,1,0,0])

[800, 800, 0, 0]
